In [82]:
using Polyhedra
using CDDLib
using LinearAlgebra
using GLPK
using HiGHS
using Statistics

In [83]:
function sample_random_direction(num_samples, n)
    v = randn(num_samples, n)
    return v ./ norm.(eachrow(v))
end

# Provides bounds for t in a parameterized line v + td inside the polytope defined by Ax <= b
# A is m×n, b is m×1, v is n-element vector (current point),
# d is n-element direction vector.
# Returns (t_min, t_max) so that for all i, A_i*(v + t*d) <= b_i.
function get_bounds(A, b, v, d)
    unscaled = (b .- (A * v))[:, 1]
    scale_factors = (A * d)[:, 1]

    pos_mask = scale_factors .> 0
    neg_mask = scale_factors .< 0
    zero_mask = scale_factors .== 0

    if any(zero_mask .& (unscaled .< 0))
        return (NaN, NaN)
    end

    t_max = Inf
    if any(pos_mask)
        t_max = minimum(unscaled[pos_mask] ./ scale_factors[pos_mask])
    end

    t_min = -Inf
    if any(neg_mask)
        t_min = maximum(unscaled[neg_mask] ./ scale_factors[neg_mask])
    end

    return (t_min, t_max)
end


function hit_and_run_start(A, b, v, n)
    n_size = length(v)
    samples = zeros(n, n_size)

    sample_count = 0
    while sample_count < n
        # Sample from the unit hypersphere
        d = randn(n_size)
        d /= norm(d)

        # Get bounds for t
        t_min, t_max = get_bounds(A, b, v, d)

        # Skip if bounds are invalid or infeasible
        if isnan(t_min) || isnan(t_max) || t_min > t_max || isinf(t_min) || isinf(t_max)
            continue  # Try a new direction
        end

        # Sample t uniformly in [t_min, t_max]
        t = rand() * (t_max - t_min) + t_min
        v = v + t * d
        sample_count += 1
        samples[sample_count, :] = v
    end

    return samples
end

function get_initial_sample(A, b, lib)
    p = polyhedron(hrep(A, b), lib)
    center, radius = chebyshevcenter(p)
    return center
end

function hit_and_run(A, b, n, lib; burnin=100)
    # Obtain initial interior point
    v_0 = get_initial_sample(A, b; lib=lib)

    # Run burn-in phase
    burnin_samples = hit_and_run_start(A, b, v_0, burnin)

    # Produce samples
    v = burnin_samples[end, :]
    return hit_and_run_start(A, b, v, n)
end

hit_and_run (generic function with 2 methods)

In [84]:
function unit_hypercube_constraints(d)
    A = [ Matrix(I, d, d); -Matrix(I, d, d) ]
    b = [ ones(d); zeros(d) ]
    return A, b
end

unit_hypercube_constraints (generic function with 1 method)

In [85]:
A, b = unit_hypercube_constraints(4)
lib = DefaultLibrary{Float64}(HiGHS.Optimizer)
samples = hit_and_run(A, b, 1000, lib; burnin=100)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 5 cols, 16 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-8); columns 0(-5); elements 0(-16) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.0000000000e-01
HiGHS run time      :       

1000×4 Matrix{Float64}:
 0.592875  0.998295  0.75927    0.920722
 0.743128  0.936543  0.708301   0.98924
 0.55217   0.56401   0.345148   0.360383
 0.551253  0.598525  0.250539   0.349016
 0.527457  0.601042  0.206539   0.333824
 0.32617   0.40359   0.378567   0.965014
 0.566948  0.688436  0.464642   0.952298
 0.575453  0.672722  0.435004   0.964181
 0.862806  0.43561   0.0803432  0.250683
 0.864713  0.435204  0.066323   0.255441
 0.875423  0.447885  0.0341037  0.276877
 0.923515  0.722745  0.138593   0.25106
 0.904142  0.88064   0.0620393  0.0321585
 ⋮                              
 0.561892  0.459275  0.588356   0.534224
 0.303053  0.38901   0.155343   0.691318
 0.283171  0.324797  0.195506   0.621205
 0.125685  0.515058  0.51574    0.769477
 0.1576    0.521593  0.476687   0.789719
 0.305482  0.245983  0.339379   0.776228
 0.254314  0.211334  0.352022   0.798318
 0.231369  0.352135  0.466734   0.630397
 0.22195   0.34428   0.469202   0.518293
 0.857323  0.233589  0.670743   0.413694
 